In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime
from datetime import timedelta
import seaborn as sns

import statsmodels.api as sm
from statsmodels.tsa.stattools import acf, pacf
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from statsmodels.tsa.seasonal import seasonal_decompose

%matplotlib inline

In [ ]:
train = pd.read_csv('../data/train_clean.csv',  index_col='Date', parse_dates=['Date'])
spray = pd.read_csv('../data/updated_spray.csv', index_col='Date', parse_dates=['Date'])
weather = pd.read_csv('../data/clean_weather.csv', index_col='Date', parse_dates=['Date'])

In [ ]:
weather.drop('Unnamed: 0', axis=1, inplace=True)
train.drop('Unnamed: 0', axis=1, inplace=True)

In [ ]:
train.shape

In [ ]:
weather.shape

In [ ]:
weather_stn1 = weather[weather['Station']==1]
weather_stn2 = weather[weather['Station']==2]
weather_stn1 = weather_stn1.drop('Station', axis=1)
weather_stn2 = weather_stn2.drop('Station', axis=1)
weather = weather_stn1.merge(weather_stn2, on='Date')

In [ ]:
weather.shape

In [ ]:
weather.columns

In [ ]:
str(weather['Sunset_x'].iloc[0])

In [ ]:
from datetime import datetime

In [ ]:
def day_length(row):
    sunset = row['Sunset_x']
    sunrise = row['Sunrise_x']
    if sunset % 100 == 60:
        sunset = sunset + 40
        sunset = str(sunset)
        sunrise = str(sunrise)
    else:
        sunset = str(sunset)
        sunrise = str(sunrise)
    
    x = datetime.strptime(sunset, '%H%M') - datetime.strptime(sunrise, '%H%M')
    return x.seconds
    # parse into datetime
    # find the difference
    # format  for output
    # return

In [ ]:
weather['Day_length'] = weather.apply(day_length, axis=1)

In [ ]:
weather['Tmax'] = weather.apply(lambda x: np.mean([x['Tmax_x'],x['Tmax_y']]), 1)

In [ ]:
weather['Tmin'] = weather.apply(lambda x: np.mean([x['Tmin_x'],x['Tmin_y']]),1)

In [ ]:
weather['Tavg'] = weather.apply(lambda x: np.mean([x['Tavg_x'],x['Tavg_y']]),1)

In [ ]:
weather['ResultSpeed'] = weather.apply(lambda x: np.mean([x['ResultSpeed_x'],x['ResultSpeed_y']]),1)

In [ ]:
weather['ResultDir'] = weather.apply(lambda x: np.mean([x['ResultDir_x'],x['ResultDir_y']]),1)

In [ ]:
weather['AvgSpeed'] = weather.apply(lambda x: np.mean([x['AvgSpeed_x'],x['AvgSpeed_y']]),1)

In [ ]:
weather['Sunset'] = weather['Sunset_x']

In [ ]:
weather['Sunrise'] = weather['Sunrise_x']

In [ ]:
weather['Heat'] = weather.apply(lambda x: np.mean([x['Heat_x'],x['Heat_y']]),1)

In [ ]:
weather['Depart'] = weather['Depart_x']

In [ ]:
weather['DewPoint'] = weather.apply(lambda x: np.mean([weather['DewPoint_x'],weather['DewPoint_y']]),1)

In [ ]:
weather['WetBulb'] = weather.apply(lambda x: np.mean([x['WetBulb_x'],x['WetBulb_y']]),1)

In [ ]:
weather['Cool'] = weather.apply(lambda x: np.mean([x['Cool_x'],x['Cool_y']]),1)

In [ ]:
weather['CodeSum'] = weather['CodeSum_x']

In [ ]:
weather['PrecipTotal'] = weather.apply(lambda x: np.mean([x['PrecipTotal_x'],x['PrecipTotal_y']]),1)

In [ ]:
weather['StnPressure'] = weather.apply(lambda x: np.mean([x['StnPressure_x'],x['StnPressure_y']]),1)

In [ ]:
weather.columns

In [ ]:
weather.drop([
         'SeaLevel_x', 
         'SeaLevel_y',
         'Tavg_x',
         'Tavg_y',
         'ResultSpeed_x',
         'ResultSpeed_y',
         'ResultDir_x',
         'ResultDir_y',
         'AvgSpeed_x',
         'AvgSpeed_y',
         'Heat_x',
         'Heat_y',
         'Tmax_x',
         'Tmax_y',
         'Tmin_x',
         'Tmin_y',
         'Sunset_y',
         'Sunrise_y',
         'Depart_y',
         'DewPoint_x',
         'DewPoint_y',
         'WetBulb_x',
         'WetBulb_y',
         'Cool_x',
         'Cool_y',
         'Sunrise_x',
         'Sunset_x',
         'CodeSum_x',
         'CodeSum_y',
         'Depth_x',
         'Depth_y',
         'PrecipTotal_x',
         'PrecipTotal_y',
         'StnPressure_x',
         'StnPressure_y',
         'Depart_x',
         'Depart_y',
         'DewPoint_x',
         'DewPoint_y'], 1, inplace=True)

In [ ]:
weather.columns

In [ ]:
train.columns

In [ ]:
df = pd.merge(train, weather, on = 'Date')

In [ ]:
df.shape

In [ ]:
df.columns

In [ ]:
df.to_csv('../data/dataframe.csv')